In [1]:
import numpy as np
import pandas as pd
from abc import abstractmethod
from lib import CovarianceModel, Hemisphere, df_residuals, pretty_cov

In [2]:
train_set = pd.read_csv('train.csv')
val_set = pd.read_csv('val.csv')
test_set = pd.read_csv('test.csv')

In [3]:
class GaussianProcessCovBaseModel(CovarianceModel):
    """
    This model estimates var[long] and var[lat] and cov[long, lat] independently with the same technique

    Train:
    * Create a space with a dimension for each latent variable, and a dimension for each quantity we're estimating
    * For each point in the training set, add a vector to the space with the latent variables and where var[X] is estimated by
      the squared prediction error in X (ie we're assuming a z-score of 1 treating X as a univariate normal distribution with 0 mean)
      and cov[X, Y] is estimated by the product of errors in X and Y (because var[X]=E[X^2] and cov[X,Y]=E[X]E[Y] with a 0 mean)

    Evaluate:
    * We are given some latent variables
    * Compute the average of all the var[X]s in the space, weighted by a covariance function centred on the latent variables

    Then we return the covariance matrix with the average var[X]s on the diagonal and 0s everywhere else
    """

    def __init__(self):
        self.space = None

    def train(self, train_set: pd.DataFrame):
        residuals = df_residuals(train_set).T
        self.scale = np.array([0.5, 1, 3])/5
        self.space = np.column_stack((
            train_set.long.to_numpy(),
            train_set.lat.to_numpy(),
            train_set.intensity.to_numpy(),
            residuals[0]**2, # var[long]
            residuals[1]**2, # var[lat]
            residuals[0]*residuals[1], # cov[long, lat]
        ))
        self.space[:,:3] *= self.scale
        print("Finished training")

    @property
    def latent_space_scaled(self) -> np.ndarray:
        return self.space[:,:3]

    # this calculates the covariance in the regression variables between different latent points
    # eg cov[var[long] at latent point 1, var[long] at latent point 2]
    # it does this at every point in the space to get a non-normalised weight vector
    # where each entry is weight_i = cov[var[long] at latent point i, var[long] at latent point we're evaluating at]
    # note that it's a different covariance! it's covariance in the Gaussian Process sense
    @abstractmethod
    def covariance_function(self, latent_scaled: np.ndarray) -> np.ndarray:
        pass

    def estimate(self, long: float, lat: float, intensity: float) -> np.ndarray:
        # create a list of weights for each point in the space
        # the weight is the Gaussian filter / window centred on the given long/lat/intensity
        latent_scaled = np.array([long, lat, intensity]) * self.scale
        weights = self.covariance_function(latent_scaled)
        weights /= np.sum(weights)
        # compute the weighted average of the var[X]s
        weighted_var_long = np.sum(weights * self.space[:,3])
        weighted_var_lat = np.sum(weights * self.space[:,4])
        weighted_cov_long_lat = np.sum(weights * self.space[:,5])
        return np.array([[weighted_var_long, weighted_cov_long_lat], [weighted_cov_long_lat, weighted_var_lat]])

In [4]:
class GaussianProcessGaussianFilterCovModel(GaussianProcessCovBaseModel):
    def covariance_function(self, latent_scaled: np.ndarray) -> np.ndarray:
        return np.exp(-0.5 * np.sum((self.latent_space_scaled - latent_scaled)**2, axis=1))

GaussianProcessGaussianFilterCovModel.assess(train_set, test_set)

Finished training


100%|██████████| 21544/21544 [02:13<00:00, 161.57it/s]

GaussianProcessGaussianFilterCovModel:
  log likelihood: -83665
  log geo mean likelihood: -3.883
  geo mean p density: 0.02058


In [5]:
class GaussianProcessExponentialFilterCovModel(GaussianProcessCovBaseModel):
    def covariance_function(self, latent_scaled: np.ndarray) -> np.ndarray:
        return np.exp(-np.linalg.norm(self.latent_space_scaled - latent_scaled, axis=1)*2)

GaussianProcessExponentialFilterCovModel.assess(train_set, test_set)

Finished training


100%|██████████| 21544/21544 [02:15<00:00, 159.38it/s]

GaussianProcessExponentialFilterCovModel:
  log likelihood: -83699
  log geo mean likelihood: -3.885
  geo mean p density: 0.02055


In [6]:
from sklearn.neighbors import NearestNeighbors

class NearestNeighborsCovModel(GaussianProcessCovBaseModel):
    def train(self, train_set: pd.DataFrame):
        super().train(train_set)
        self.neigh = NearestNeighbors(n_neighbors=2000).fit(self.space[:,:3])
        print("Finished fitting neighbors")

    def covariance_function(self, latent_scaled: np.ndarray) -> np.ndarray:
        # the weights are uniform across the nearest neighbors
        indices = self.neigh.kneighbors([latent_scaled], return_distance=False)[0]
        weights = np.zeros(len(self.space))
        weights[indices] = 1 / len(indices)
        return weights

NearestNeighborsCovModel.assess(train_set, test_set)

Finished training
Finished fitting neighbors


100%|██████████| 21544/21544 [00:37<00:00, 581.82it/s]

NearestNeighborsCovModel:
  log likelihood: -84597
  log geo mean likelihood: -3.927
  geo mean p density: 0.01971
